In [3]:
%matplotlib inline

# Algorithm 1

In [1]:
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

## Define functions

In [80]:
def f(param, y, scale=1):
    return scipy.stats.norm.pdf(y, loc = param, scale = scale)

In [81]:
def G_tilde(t_p, nu, w_tilde):
    i = np.argmax(w_tilde)
    return nu[i]

def G(t_p, teta, w):
    return sum([teta[i] * w[i] for i in range(len(teta))]) / sum(w)


In [82]:
def F_tilde(nu_bar, epsilon_p=0, X_bar=0, t_p=0):
    return np.linspace(nu_bar - 350 * epsilon_p / K_(N), nu_bar + 350 * epsilon_p / K_(N), num=N_tilde) # change for larger dimension

def F(nu_bar, xi_p=0, X_bar=0, t_p=0):
    return np.linspace(nu_bar - 350 * xi_p / K_(N), nu_bar + 350 * xi_p / K_(N), num=N)

In [83]:
def t_(p, former_t_p):
    return former_t_p + max(int((k**(-2) - 1) * former_t_p) + 1, t_1)

def epsilon_(p):
    # d ?
    return epsilon_0 * min(1, (p**(-1) * rho * np.log(p + 1))**(1 / (d + beta)))

def c_(p):
    if p==0:
        return 1
    return min(((1 + k) / (2 * k))**p, p**((1 + epsilon) / 2))

def K_(N):
    k = 1
    while k**d <= N:
        k += 1
    return k - 1

## Inputs and Initialization

In [89]:
####
#Inputs
#####

N = 20
M = 10
t_1 = 10

epsilon = 1
epsilon_0 = 1
beta = 1

k = 0.5

rho = 3

theta = np.random.uniform(-100, 100, N)
nu = np.random.uniform(-100, 100, N + M)


########## 

nb_iterations = 10000

theta_star = 0
scale = 20

Y = np.random.normal(theta_star, scale, nb_iterations)

d = 1 # ??

########
#Define
######

N_tilde = N + M
X_tilde = (N, M, k, beta)

t_0 = 0
t_p = t_1

c_0 = 1

#####
#Initilization
#####

w = (1/N) * np.ones(N)
w_tilde = (1/N_tilde) * np.ones(N_tilde)

q_0 = 0

xi = 1
p = 1

# Animations

#### Without perturbation :

In [141]:
%matplotlib
fig = plt.figure()
ax = fig.add_subplot(111)
for t in range(1, nb_iterations):
    
    sum_w = np.sum(w)
    normalized_w = [w[i] / sum_w for i in range(N)]
    #plotting :
    if t % 1 == 0:
        x = [-100.2, 100.2, 100.2, -100.2]
        y = [-0.1, -0.1, 1, 1]
        plt.fill(x, y, "white", zorder= 2 * t -1)
        plt.scatter(theta, np.zeros(N), c="red", zorder = 2 * t)
        plt.bar(theta, normalized_w, zorder = 2 * t)
        fig.canvas.draw()
        plt.pause(1)
        
    ## Bayes updates
    w_tilde = [w_tilde[i] * f(nu[i], Y[t]) for i in range(N_tilde)]
    w = [w[i] * f(theta[i], Y[t]) for i in range(N)]
    
    
    


Using matplotlib backend: MacOSX


KeyboardInterrupt: 

#### just $\eta(t)$

In [88]:
%matplotlib
fig = plt.figure()
ax = fig.add_subplot(111)

# plot the real law
x_density = np.linspace(-100,100,100) # 100 linearly spaced numbers
y_density = 15 * scipy.stats.norm.pdf(x_density, loc=theta_star, scale=scale) # computing the values of sin(x)/x

for t in range(1, nb_iterations):
    if t%100 == 0:
        print(t)
    
    if t == t_p + 1:
    # Perturbation
        print("PERTURBATION")
        print(epsilon_(p))
        nu_bar = G_tilde(t_p, nu, w_tilde)
        nu = F_tilde(nu_bar, epsilon_(p))
        w_tilde = (1/N_tilde) * np.ones(N_tilde)
        p += 1
        t_p = t_(p, t_p)
        

    
    
    
    
    #plotting :
    if t % 10 == 0:

        plt.clf()
        plt.plot(x_density, y_density) #plot real density function
        plt.scatter(nu, np.zeros(N_tilde), c="red")
        plt.bar(nu, w_tilde)
        plt.scatter([Y[t]], [15 * scipy.stats.norm.pdf(Y[t], loc=theta_star, scale=scale)], marker="*") # plot observation
        plt.pause(0.001)
        
    ## Bayes updates<
    w_tilde = [w_tilde[i] * f(nu[i], Y[t]) for i in range(N_tilde)]
    sum_w_tilde = np.sum(w_tilde)
    w_tilde = [w_tilde[i] / sum_w_tilde for i in range(N_tilde)] # normalize

    
    
    
    


Using matplotlib backend: MacOSX


/Users/anne/anaconda/envs/modal/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars


PERTURBATION
1
PERTURBATION
1
100
PERTURBATION
1
200
300
400
500
600
PERTURBATION
1
700


KeyboardInterrupt: 

$\pi(t)$ without auxiliary support

In [86]:
%matplotlib
fig = plt.figure()
ax = fig.add_subplot(111)

q = 0

# plot the real law
x_density = np.linspace(-100,100,100) # 100 linearly spaced numbers
y_density = 15 * scipy.stats.norm.pdf(x_density, loc=theta_star, scale=scale) # computing the values of sin(x)/x

for t in range(1, nb_iterations):

    if t == t_p + 1:
        # Perturbation
        print("PERTURBATION")
        theta_bar = G(t_p, theta, w)
        q += 1
        xi = k * (c_(q) / c_(q - 1)) * xi
        theta_hat = theta_bar
        theta = F(theta_hat, xi)
        w = np.ones(N)
        p += 1
        t_p = t_(p, t_p)
        
    
    
    

    

    if t % 10 == 0:
        plt.clf()
        plt.plot(x_density, y_density) #plot real density function
        plt.scatter(theta, np.zeros(N), c="red")
        plt.scatter([Y[t]], [15 * scipy.stats.norm.pdf(Y[t], loc=theta_star, scale=scale)], marker="*") # plot observation
        plt.bar(theta, w)
        fig.canvas.draw()
        plt.pause(0.001)
        
    ## Bayes updates
        ## Bayes updates<
    w = [w[i] * f(theta[i], Y[t]) for i in range(N)]
    sum_w = np.sum(w)
    w = [w[i] / sum_w for i in range(N)] # normalize
    

    
    


Using matplotlib backend: MacOSX
PERTURBATION


/Users/anne/anaconda/envs/modal/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in double_scalars


PERTURBATION
PERTURBATION


KeyboardInterrupt: 

#### With perturbation

In [90]:
%matplotlib
fig = plt.figure()
ax = fig.add_subplot(111)

q = 0

for t in range(1, nb_iterations):

    if t == t_p + 1:
        # Perturbation
        print("PERTURBATION")
        print(epsilon_(p))
        theta_bar = G(t_p, theta, w)
        nu_bar = G_tilde(t_p, nu, w_tilde)
        if (theta_bar - nu_bar) <= 2 * epsilon_(p):
            q += 1
            xi = k * (c_(q) / c_(q - 1)) * xi
            theta_hat = theta_bar
        else:
            q = 1
            xi = epsilon_(p)
            theta_hat = nu_bar
        
        theta = F(theta_hat, xi)
        nu = F_tilde(nu_bar, epsilon_(p))
        w = np.ones(N)
        w_tilde = np.ones(N_tilde)
        p += 1
        t_p = t_(p, t_p)
        
    
    
    

    
########## plotting
    if t % 10 == 0:
        plt.clf()
        plt.plot(x_density, y_density) #plot real density function
        plt.scatter(theta, np.zeros(N), c="red")
        plt.scatter([Y[t]], [15 * scipy.stats.norm.pdf(Y[t], loc=theta_star, scale=scale)], marker="*") # plot observation
        plt.bar(theta, w)
        fig.canvas.draw()
        plt.pause(0.001)
        
######## Exporting
        
    ## Bayes updates
    w_tilde = [w_tilde[i] * f(nu[i], Y[t]) for i in range(N_tilde)]
    w = [w[i] * f(theta[i], Y[t]) for i in range(N)]
    sum_w = np.sum(w)
    w = [w[i] / sum_w for i in range(N)] # normalize
    sum_w_tilde = np.sum(w_tilde)
    w_tilde = [w_tilde[i] / sum_w_tilde for i in range(N_tilde)] # normalize
    
    
    


Using matplotlib backend: MacOSX


/Users/anne/anaconda/envs/modal/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in double_scalars
/Users/anne/anaconda/envs/modal/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in double_scalars


PERTURBATION
1
PERTURBATION
1
PERTURBATION
1
PERTURBATION
1


KeyboardInterrupt: 

# Convergence rates

In [ ]:
def check_concentration(speed, N_samples_for_estimation, n, d, sig_ep, sig_beta, step=30, number_of_trials_per_step=10):
    df = pd.DataFrame(columns=["n", "proba"])
    Beta_star = sig_beta*np.random.randn(d) #real value of Beta
    for trial in range(number_of_trials_per_step):
        for i in range(3, n, step): # begins at 3 because we remove 2 lines
            [X, Y] = generate_samples(sig_ep, i, d,Beta_star)
            posteriorMeanBeta, posteriorCovBeta = calculate_posterior_beta(X, Y, d, i)
            count_inside_ball = 0
            for k in range(N_samples_for_estimation):
                count_inside_ball += np.linalg.norm(np.random.multivariate_normal(posteriorMeanBeta, posteriorCovBeta) - Beta_star)< 1 / speed(i)
            df2 = pd.DataFrame([[i, count_inside_ball / N_samples_for_estimation]],columns=['n', 'proba'])
            df = pd.concat([df,df2])
    sns.barplot(x='n', y="proba", data = df)

### Just $\pi(t)$ without auxiliary support

In [38]:
def pi_without_aux(nb_iterations, theta_star):
    ####
    #Inputs
    #####

    N = 20
    M = 10
    t_1 = 10

    epsilon = 1
    epsilon_0 = 1
    beta = 1

    k = 0.5

    rho = 3

    theta = np.random.uniform(-100, 100, N)
    nu = np.random.uniform(-100, 100, N + M)

    ########## 
    scale = 20

    Y = np.random.normal(theta_star, scale, nb_iterations)

    d = 1 # ??
    ########
    #Define
    ######
    N_tilde = N + M
    X_tilde = (N, M, k, beta)

    t_0 = 0

    c_0 = 1
    #####
    #Initilization
    #####
    w = (1/N) * np.ones(N)
    w_tilde = (1/N_tilde) * np.ones(N_tilde)
    
        t_p = t_1


    q_0 = 0

    xi = 1
    p = 1

    q = 0
    
    
    ###BEGIN
    
    list_of_distrib = []
    list_of_support = []
    

    
    for t in range(1, nb_iterations):

        if t == t_p + 1:
            # Perturbation
            print("PERTURBATION")
            theta_bar = G(t_p, theta, w)
            q += 1
            xi = k * (c_(q, k, epsilon) / c_(q - 1, k, epsilon)) * xi
            theta_hat = theta_bar
            theta = F(theta_hat, xi)
            w = np.ones(N)
            p += 1
            t_p = t_(p, t_p)




        ## Bayes updates
            ## Bayes updates<
        w = [w[i] * f(theta[i], Y[t]) for i in range(N)]
        sum_w = np.sum(w)
        w = [w[i] / sum_w for i in range(N)] # normalize
        
        list_of_distrib.append(w) 
        list_of_support.append(theta)
        
    return list_of_support, list_of_distrib



    


In [105]:
class Algorithm:
    
    def __init__(self, N=20, M=10, t_1=10, epsilon=1, epsilon_0=1, beta=1, k=0.5, rho=3, d=1, theta_star=0):
        # Set up the parameters for future runs
        self.N = N
        self.M = M
        self.t_1 = t_1
        self.epsilon = epsilon
        self.epsilon_0 = epsilon_0
        self.beta = beta
        self.k = k
        self.rho = rho
        self.d = 1
        self.N_tilde = N + M
        self.X_tilde = (N, M, k, beta)
        self.t_0 = 0
        self.c_0 = 1
        
        
    def f(self, param, y, scale=1):
        return scipy.stats.norm.pdf(y, loc = param, scale = scale)
    
    def G_tilde(self, t_p, nu, w_tilde):
        i = np.argmax(w_tilde)
        return nu[i]

    def G(self, t_p, teta, w):
        return sum([teta[i] * w[i] for i in range(len(teta))]) / sum(w)
    
    def F_tilde(self, nu_bar, epsilon_p=0, X_bar=0, t_p=0):
        return np.linspace(nu_bar - 350 * epsilon_p / self.K_(self.N_tilde), nu_bar + 350 * epsilon_p / self.K_(self.N_tilde), num=self.N_tilde) # change for larger dimension

    def F(self, nu_bar, xi_p=0, X_bar=0, t_p=0):
        return np.linspace(nu_bar - 350 * xi_p / self.K_(self.N), nu_bar + 350 * xi_p / self.K_(self.N), num=self.N)
    
    def t_(self, p, former_t_p):
        return former_t_p + max(int((self.k**(-2) - 1) * former_t_p) + 1, self.t_1)

    def epsilon_(self, p):
        # d ?
        return self.epsilon_0 * min(1, (p**(-1) * self.rho * np.log(p + 1))**(1 / (self.d + self.beta)))

    def c_(self, p):
        if p==0:
            return 1
        return min(((1 + self.k) / (2 * self.k))**p, p**((1 + self.epsilon) / 2))

    def K_(self, N):
        k = 1
        while k**self.d <= self.N:
            k += 1
        return k - 1
    
    
    def plot(self, time_interval, theta_star, scale, observation, theta, w, fig):
        # plot the real law
        x_density = np.linspace(-100,100,100) 
        y_density = 15 * scipy.stats.norm.pdf(x_density, loc=theta_star, scale=scale) 
        plt.clf()
        plt.plot(x_density, y_density) #plot real density function
        plt.scatter(theta, np.zeros(self.N), c="red")
        plt.scatter([observation], [15 * scipy.stats.norm.pdf(observation, loc=theta_star, scale=scale)], marker="*") # plot observation
        plt.bar(theta, w)
        fig.canvas.draw()
        plt.pause(time_interval)
        
    def bayes_update(self, distrib, support, f, scale, observation):
        distrib = [distrib[i] * self.f(support[i], observation, scale=scale) for i in range(len(support))]
        sum_distrib = np.sum(distrib)
        distrib = [distrib[i] / sum_distrib for i in range(len(support))] # normalize
        
        return distrib



    
    def animate_pi_without_support(self, nb_iterations=10000, time_interval=0.001, theta_star=0, scale=20):
        fig = plt.figure()
        ax = fig.add_subplot(111)
        #Initialize
        theta = np.random.uniform(-100, 100, self.N)
        nu = np.random.uniform(-100, 100, self.N + self.M)
        Y = np.random.normal(theta_star, scale, nb_iterations)
        w = (1/self.N) * np.ones(self.N)
        w_tilde = (1/self.N_tilde) * np.ones(self.N_tilde)
    
        t_p = self.t_1
        xi = 1
        p = 1
        q = 0
        for t in range(1, nb_iterations):
            if t == t_p + 1:
                # Perturbation
                print("PERTURBATION")
                theta_bar = self.G(t_p, theta, w)
                q += 1
                xi = self.k * (self.c_(q) / self.c_(q - 1)) * xi
                theta_hat = theta_bar
                theta = self.F(theta_hat, xi)
                w = np.ones(self.N)
                p += 1
                t_p = self.t_(p, t_p)
                

        ########## plotting
            if t % 10 == 0:
                self.plot(time_interval, theta_star, scale, observation = Y[t], theta=theta, w=w, fig=fig)

            ## Bayes updates
            w = self.bayes_update(w, theta, self.f, scale, Y[t])
            
    def animate_pi(self, nb_iterations=10000, time_interval=0.001, theta_star=0, scale=20):
        fig = plt.figure()
        ax = fig.add_subplot(111)
        #Initialize
        theta = np.random.uniform(-100, 100, self.N)
        nu = np.random.uniform(-100, 100, self.N + self.M)
        Y = np.random.normal(theta_star, scale, nb_iterations)
        w = (1/self.N) * np.ones(self.N)
        w_tilde = (1/self.N_tilde) * np.ones(self.N_tilde)
    
        t_p = self.t_1
        xi = 1
        p = 1
        q = 0
        for t in range(1, nb_iterations):
            if t == t_p + 1:
                # Perturbation
                print("PERTURBATION")
                theta_bar = self.G(t_p, theta, w)
                nu_bar = self.G_tilde(t_p, nu, w_tilde)
                if (theta_bar - nu_bar) <= 2 * self.epsilon_(p):
                    q += 1
                    xi = self.k * (self.c_(q) / self.c_(q - 1)) * xi
                    theta_hat = theta_bar
                else:
                    q = 1
                    xi = self.epsilon_(p)
                    theta_hat = nu_bar
                theta = self.F(theta_hat, xi)
                nu = F_tilde(nu_bar, epsilon_(p))
                w = np.ones(self.N)
                w_tilde = np.ones(N_tilde)
                p += 1
                t_p = self.t_(p, t_p)
        
    
        ########## plotting
            if t % 10 == 0:
                self.plot(time_interval, theta_star, scale, observation = Y[t], theta=theta, w=w, fig=fig)

            ## Bayes updates
            w = self.bayes_update(w, theta, self.f, scale, Y[t])
            w_tilde = self.bayes_update(w_tilde, nu, self.f, scale, Y[t])


    
    def run(nb_iterations, theta_star=0, scale=20):
        #Initialize
        theta = np.random.uniform(-100, 100, N)
        nu = np.random.uniform(-100, 100, N + M)
        Y = np.random.normal(theta_star, scale, nb_iterations)
        w = (1/N) * np.ones(N)
        w_tilde = (1/N_tilde) * np.ones(N_tilde)
    
        t_p = t_1
        xi = 1
        p = 1
        q = 0
    
    
    
        

In [106]:
test = Algorithm()

In [107]:
%matplotlib
test.animate_pi()

Using matplotlib backend: MacOSX
PERTURBATION
PERTURBATION
PERTURBATION
PERTURBATION


KeyboardInterrupt: 

In [ ]:
def wassertein(nb_iterations, theta_star=0):
    list_of_support, list_of_distrib = pi_without_aux(nb_iterations, theta_star)
    list_of_distances = [scipy.stats.wasserstein_distance(list_of_support[i], [theta_star], list_of_distrib[i], [1]) for i in range(nb_iterations)]
    plt.plot(x=[i in range(nb_iterations)], y=list_of_distances)
        

In [40]:
wassertein(1000)

PERTURBATION


/Users/anne/anaconda/envs/modal/lib/python3.7/site-packages/ipykernel_launcher.py:79: RuntimeWarning: invalid value encountered in double_scalars


NameError: name 'N' is not defined

In [ ]:
#%matplotlib

fig, ax = plt.subplots()
xdata, ydata = [], []
density, = plt.plot([], [], 'ro', animated=True)
support = plt.scatter([], [], animated=True)
emp_density = plt.bar(nu, w_tilde, animated=True)
observation = plt.scatter([], [], marker="*", animated=True)
# plot the real law
x_density = np.linspace(-100,100,100)
y_density = 15 * scipy.stats.norm.pdf(x_density, loc=theta_star, scale=scale) 

lines = [density, support, emp_density, observation]

#print([x for x in dir(support) if 'set_' in x])

def init():
    ax.set_xlim(-100, 100)
    ax.set_ylim(-1, 1)

    lines[0].set_data([], [])
    lines[1].set_offsets([])
    for i, b in enumerate(lines[2]):
        print(b)
        b.set_height(w_tilde[i])
    lines[3].set_offsets([])


    return lines

def update(t):
    
    if t == t_p + 1:
    # Perturbation
        print("PERTURBATION")
        print(epsilon_(p))
        nu_bar = G_tilde(t_p, nu, w_tilde)
        nu = F_tilde(nu_bar, epsilon_(p))
        w_tilde = (1/N_tilde) * np.ones(N_tilde)
        lines[2] = plt.bar(nu, w_tilde, animated=True)
        p += 1
        t_p = t_(p, t_p)
        
    #plotting :
    if t % 1 == 0:
        lines[0].set_data(x_density, y_density) #plot real density function
        lines[1].set_data(nu, np.zeros(N_tilde), c="red")
        for i, b in enumerate(emp_density):
            b.set_height(w_tilde[i])
        lines[3].set_data([Y[t]], [15 * scipy.stats.norm.pdf(Y[t], loc=theta_star, scale=scale)], marker="*") # plot observation

        
    ## Bayes updates
    w_tilde = [w_tilde[i] * f(nu[i], Y[t]) for i in range(N_tilde)]
    sum_w_tilde = np.sum(w_tilde)
    w_tilde = [w_tilde[i] / sum_w_tilde for i in range(N_tilde)] # normalize
        
    return lines
        

ani = FuncAnimation(fig, update, frames=np.linspace(0, 2*np.pi, 128),
                    init_func=init, blit=True)
plt.show()

In [92]:
%history

import scipy.stats
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
class Algorithm:
    
    def __init__(self, N=20, M=10, t_1=10, epsilon=1, beta=1, k=0.5, rho=3, d=1, theta_star=0):
        # Set up the parameters for future runs
        self.N = N
        self.M = M
        self.t_1 = t_1
        self.epsilon = 1
        self.beta = beta
        self.k = k
        self.rho = rho
        self.d = 1
        self.N_tilde = N + M
        self.X_tilde = (N, M, k, beta)
        self.t_0 = 0
        self.c_0 = 1
        
        
    def f(param, y, scale=1):
        return scipy.stats.norm.pdf(y, loc = param, scale = scale)
    
    def G_tilde(self, t_p, nu, w_tilde):
        i = np.argmax(w_tilde)
        return nu[i]

    def G(self, t_p, teta, w):
        return sum([teta[i] * w[i] for i in range(len(teta))]) / sum(w)
    
    def F_tilde(self, nu_bar, epsilon_p=0, X_bar=0, t_p=0):
        return np.linspace(nu_